### Import libraries

In [1]:
# import necessary libraries
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

%matplotlib inline

### Set device

In [2]:
if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu"  
device = torch.device(dev)

### Define classes and functions

In [3]:
def func(x):
    return (1.2 * np.sin(np.pi * x) - np.cos(2.4 * np.pi * x))

In [4]:
class net(nn.Module):
    # define the neural network structure
    def __init__(self, n_hidden, device):
        super(net, self).__init__()
        self.layer1 = nn.Linear(1, n_hidden).to(torch.device(device))
        self.layer2 = nn.Linear(n_hidden, 1).to(torch.device(device))

    # define the data process inside our layer
    def forward(self, x, device):
        x = torch.tanh(self.layer1(x)).to(device)
        x = self.layer2(x).to(device)
        return x
    
    def train(self, train_inputs, train_labels, 
              val_inputs, val_labels, 
              device='cpu', train_type='gd', 
              num_max_epoch=1000, lr=0.01, 
              stop_val=0.01):
        self.train_loss = []
        self.val_loss = []
        
        criterion = nn.MSELoss()
        optimizer = optim.SGD(self.parameters(), lr)
        
        if(train_type == 'gd'):
            # train the model
            temp_running_loss = 0.0
            running_loss = 0.0
            for epoch in range(num_max_epoch):
                # predict the output
                tprediction = self(train_inputs, device)     
                tloss = criterion(tprediction, train_labels)
                self.train_loss.append(tloss.item())
                optimizer.zero_grad()
                tloss.backward()
                optimizer.step()
                
                # validate the outputs
                vprediction = self(val_inputs, device)
                vloss = criterion(vprediction, val_labels)
                self.val_loss.append(vloss.item())
                
                # print status
                print('\rGD epoch: {}\tLoss =  {:.3f} vLoss =  {:.3f}'.format(epoch, tloss, vloss), end="")

                
        elif(train_type == 'sgd'):            
            for epoch in range(num_max_epoch):
                training_loss = 0.0
                validation_loss = 0.0
                
                for x_i, y_i in zip(train_inputs, train_labels):
                    pred_i = self(x_i, device)
                    tloss = criterion(pred_i, y_i)
                    optimizer.zero_grad()   
                    tloss.backward()         
                    optimizer.step()
                    training_loss += tloss.item()
                
                for x_i, y_i in zip(val_inputs, val_labels):
                    pred_i = self(x_i, device)
                    vloss = criterion(pred_i, y_i)
                    validation_loss += vloss.item()
                
                training_loss = training_loss/len(train_inputs)
                validation_loss = validation_loss/len(val_inputs)
                self.train_loss.append(training_loss)
                self.val_loss.append(validation_loss)
                print('\rSGD epoch: {}\tLoss =  {:.3f} vLoss =  {:.3f}'.format(epoch, training_loss, validation_loss), end="") 
                    
                    
        elif(train_type == 'mbgd'):
            batch_size = 16
            train_n_batches = int(len(train_inputs) / batch_size) 
            val_n_batches = int(len(val_inputs) / batch_size) 
            print(train_n_batches)
            print(val_n_batches)
            for epoch in range(num_max_epoch):
                training_loss = 0.0
                validation_loss = 0.0
                
                for batch in range(train_n_batches):
                    batch_X, batch_y = train_inputs[batch*batch_size:(batch+1)*batch_size], train_labels[batch*batch_size:(batch+1)*batch_size]
                    prediction = self(batch_X, device)
                    tloss = criterion(prediction, batch_y)
                    training_loss += tloss.item()
                    optimizer.zero_grad()
                    tloss.backward() 
                    optimizer.step() 
                    
                for batch in range(val_n_batches):
                    batch_X, batch_y = val_inputs[batch*batch_size:(batch+1)*batch_size], val_labels[batch*batch_size:(batch+1)*batch_size]
                    prediction = self(batch_X, device)
                    vloss = criterion(prediction, batch_y)
                    validation_loss += vloss.item()
                    
                training_loss = training_loss/train_n_batches
                validation_loss = validation_loss/val_n_batches
                self.train_loss.append(training_loss)
                self.val_loss.append(validation_loss)
                print('\rMB GD epoch: {}\tLoss =  {:.3f} vLoss =  {:.3f} '.format(epoch, training_loss, validation_loss), end="")
                    
                    
    def test(self, test_inputs, test_labels, device='cpu'):
        self.test_loss = 0
        self.test_values = []
        criterion = nn.MSELoss()
        
        for x_i, y_i in zip(test_inputs, test_labels):
            pred_i = self(x_i, device)
            loss = criterion(pred_i, y_i)
            self.test_loss += loss.item()
            self.test_values.append(pred_i)

In [ ]:
def plot_lines(datas, 
               legends,
               title='Title',
               xlabel='xlabel',
               ylabel='ylabel'):
    plt.figure(figsize=(15,10))
    plt.grid()
    for i in range(len(legends)):
        plt.plot(datas[i])
        plt.legend(legends)
        plt.title(title, fontsize=30)
        plt.xlabel(xlabel, fontsize=20)
        plt.ylabel(ylabel, fontsize=20)
        plt.xticks(fontsize=15)
        plt.yticks(fontsize=15)

In [ ]:
def plot_bars(datas,
              legends, 
              title='Title',
              xlabel='xlabel',
              ylabel='ylabel'):
    
    plt.figure(figsize=(15, 10))    
    x = datas
    xi = list(range(len(legends)))

    plt.grid()
    plt.xticks(xi, legends, fontsize=15)
    plt.yticks(fontsize=15)
    plt.bar(xi, x)
    plt.title(title, fontsize=30)
    plt.xlabel(xlabel, fontsize=20)
    plt.ylabel(ylabel, fontsize=20)

In [ ]:
# def save_all(directory, trained_nets, hidden_net_sets):
#     for i, net_i in enumerate(trained_nets):
#         torch.save(net_i, f"{directory}model_100000_gd_{str(hidden_net_sets[i])}.pt")

#         with open(f'{directory}model_train_loss_100000_{str(hidden_net_sets[i])}.txt', 'w') as f:
#             for item in net_i.train_loss:
#                 f.write("%s\n" % item)

#         with open(f'{directory}model_val_loss_100000_{str(hidden_net_sets[i])}.txt', 'w') as f:
#             for item in net_i.val_loss:
#                 f.write("%s\n" % item)

def read_all(directory, epoch, hidden_net_sets, device):
    all_model = []
    all_train_loss = []
    all_val_loss = []
    
    for i, net_i in enumerate(hidden_net_sets):
        model = net(net_i * net_i, device)
#         model.load_state_dict(torch.load(f"{directory}model_100000_gd_{str(net_i)}.pt"))
        model = torch.load(f"{directory}model_{str(epoch)}_gd_{str(net_i)}.pt", map_location=device)
        all_model.append(model)
        
        f = open(f"model_train_loss_{str(epoch)}_{str(net_i)}.txt", "r")
        contents = f.read()
        new_arr = np.array(contents.splitlines(), dtype=np.float32).reshape((-1, 1))
        all_train_loss.append(new_arr)
        f.close()
        
        f = open(f"model_val_loss_{str(epoch)}_{str(net_i)}.txt", "r")
        contents = f.read()
        new_arr = np.array(contents.splitlines(), dtype=np.float32).reshape((-1, 1))
        all_val_loss.append(new_arr)
        f.close()
        
    return all_model, all_train_loss, all_val_loss

### Train models

In [ ]:
train_in = torch.Tensor(np.arange(-2, 2, 0.05).astype(np.float32)).reshape(-1, 1)
test_in = torch.Tensor(np.arange(-2, 2, 0.01).astype(np.float32)).reshape(-1, 1)
train_out = func(train_in).reshape(-1, 1).to(device)
test_out = func(test_in).reshape(-1, 1).to(device)
train_in = train_in.to(device)
test_in = test_in.to(device)

In [ ]:
trained_nets = []
train_losses = []
train_epochs = []
test_losses = []
hidden_net_sets = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 50, 100]

In [ ]:
for hidden_net in hidden_net_sets:
    network = net(hidden_net, device)
    network.train(train_in, train_out,
                  test_in, test_out,
                  device=device, train_type='gd',
                  num_max_epoch=100000,
                  lr=0.01, stop_val=0.01)
    
    trained_nets.append(network)

### Read Trained models

In [ ]:
epoch_train = 100000
trained_nets, train_losses, test_losses = read_all('./', epoch_train, hidden_net_sets, device)

### Plot models losses

In [ ]:
plot_lines([loss for loss in train_losses], 
           hidden_net_sets,
           title='Training in various number of neurons',
           xlabel='Epochs',
           ylabel='Train Loss')

### Plot train and test losses

In [ ]:
new_arr = np.array([loss[-1] for loss in train_losses])
plot_bars(new_arr.reshape(-1), 
          hidden_net_sets, 
          title='Train loss value of each neurons number',
          xlabel='Neurons',
          ylabel='Loss')

new_arr = np.array([loss[-1] for loss in test_losses])
plot_bars(new_arr.reshape(-1), 
          hidden_net_sets, 
          title='Test loss value of each neurons number',
          xlabel='Neurons',
          ylabel='Loss')

### Model testing

In [ ]:
def plot_figures(datas, 
                 legend=['Model', 'Reference'], 
                 title='Title', 
                 xlabel='xLabel', ylabel='yLabel', 
                 xi=[0, 100, 200, 300, 400, 500, 600],
                 x=[-3, -2, -1, 0, 1, 2, 3]):
    
    plt.figure(figsize=(15, 10))
    plt.grid()
    for i, data in enumerate(datas):
        plt.plot(data)
        plt.legend(legend)
        plt.title(title, fontsize=30)
        plt.xlabel(xlabel, fontsize=20)
        plt.ylabel(ylabel, fontsize=20)
        plt.xticks(xi, x, fontsize=20)
        plt.yticks(fontsize=20)

In [ ]:
input_try = test_in
# input_try = np.arange(-2, 2, 0.01, dtype=np.float32)
output_try = func(input_try).to(device)
input_try = input_try.to(device)
hidden_net_sets = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 50, 100]
for i, net_ser in enumerate(trained_nets):
    trained_nets[i].test(input_try, output_try)
    out = trained_nets[i].test_values
    out = np.array(out, dtype=np.float)
    
    plot_figures([out, output_try], 
             legend=['Reference', 'Model'],
             title=f"Test losses over a hidden layers with {hidden_net_sets[i]} neuron(s)", 
             xlabel='Points', ylabel='Outputs',
             xi=[0, 100, 200, 300, 400],
             x=[-2, -1, 0, 1, 2])

## Re-Train for detailed phase for 7-100 neurons

### Read trained models (7-100)

In [ ]:
epoch_train = 1000000
additional_hidden_net_sets = [7, 8, 9, 10, 20, 50, 100]
trained_nets, train_losses, test_losses = read_all('./', epoch_train, additional_hidden_net_sets, device)

### Plot models losses

In [ ]:
len(hidden_net_sets)

In [ ]:
plot_lines([loss for loss in train_losses], 
           additional_hidden_net_sets,
           title='Training in various number of neurons',
           xlabel='Epochs',
           ylabel='Train Loss')

In [ ]:
new_arr = np.array([loss[-1] for loss in train_losses])
plot_bars(new_arr.reshape(-1), 
          additional_hidden_net_sets, 
          title='Train loss value of each neurons number',
          xlabel='Neurons',
          ylabel='Loss')

new_arr = np.array([loss[-1] for loss in test_losses])
plot_bars(new_arr.reshape(-1), 
          additional_hidden_net_sets, 
          title='Test loss value of each neurons number',
          xlabel='Neurons',
          ylabel='Loss')

In [ ]:
input_try = test_in
# input_try = np.arange(-2, 2, 0.01, dtype=np.float32)
output_try = func(input_try).to(device)
input_try = input_try.to(device)
for i, net_ser in enumerate(trained_nets):
    trained_nets[i].test(input_try, output_try)
    out = trained_nets[i].test_values
    out = np.array(out, dtype=np.float)
    
    plot_figures([out, output_try], 
             legend=['Reference', 'Model'],
             title=f"Test losses over a hidden layers with {additional_hidden_net_sets[i]} neuron(s)", 
             xlabel='Points', ylabel='Outputs',
             xi=[0, 100, 200, 300, 400],
             x=[-2, -1, 0, 1, 2])

### Input -3 and 3

In [ ]:
hidden_net_sets = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 50, 100]
for i, net in enumerate(trained_nets):
    a, b = net(torch.tensor([-3.0]), device).item(), net(torch.tensor([3.0]), device).item()
    c, d = net(torch.tensor([-1.0]), device).item(), net(torch.tensor([1.0]), device).item()
    e = net(torch.tensor([0.0]), device).item()
    print(f"Result for {hidden_net_sets[i]} is :")
    print(f"(-3) = {round(a,3)}")
    print(f"(-1) = {round(c,2)}")
    print(f"(0) = {round(e,2)}")
    print(f"(+1) = {round(d,2)}")
    print(f"(+3) = {round(b,2)}")

In [ ]:
# input_try = test_in
input_try = np.arange(-3, 3, 0.01, dtype=np.float32).reshape(-1, 1)
output_try = func(input_try).reshape(-1, 1)
input_try = torch.Tensor(input_try).to(device)
output_try = torch.Tensor(output_try).to(device)
hidden_net_sets = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 50, 100]
for i, net_ser in enumerate(trained_nets):
    trained_nets[i].test(input_try, output_try)
    out = trained_nets[i].test_values
    out = np.array(out, dtype=np.float)
    
    plot_figures([output_try, out], 
             legend=['Reference', 'Model'],
             title=f"Test losses over a hidden layers with {hidden_net_sets[i]} neuron(s)", 
             xlabel='Points', ylabel='Outputs',
             xi=[0, 100, 200, 300, 400, 500, 600],
             x=[-3, -2, -1, 0, 1, 2, 3])

### Input -3 and 3 v2

In [ ]:
additional_hidden_net_sets

In [ ]:
for i, net in enumerate(trained_nets):
    a, b = net(torch.tensor([-3.0]), device).item(), net(torch.tensor([3.0]), device).item()
    c, d = net(torch.tensor([-1.0]), device).item(), net(torch.tensor([1.0]), device).item()
    e = net(torch.tensor([0.0]), device).item()
    print(f"Result for {additional_hidden_net_sets[i]} is :")
    print(f"(-3) = {round(a,3)}")
    print(f"(-1) = {round(c,2)}")
    print(f"(0) = {round(e,2)}")
    print(f"(+1) = {round(d,2)}")
    print(f"(+3) = {round(b,2)}")

In [ ]:
# input_try = test_in
input_try = np.arange(-3, 3, 0.01, dtype=np.float32).reshape(-1, 1)
output_try = func(input_try).reshape(-1, 1)
input_try = torch.Tensor(input_try).to(device)
output_try = torch.Tensor(output_try).to(device)
for i, net_ser in enumerate(trained_nets):
    trained_nets[i].test(input_try, output_try)
    out = trained_nets[i].test_values
    out = np.array(out, dtype=np.float)
    
    plot_figures([output_try, out], 
             legend=['Reference', 'Model'],
             title=f"Test losses over a hidden layers with {additional_hidden_net_sets[i]} neuron(s)", 
             xlabel='Points', ylabel='Outputs',
             xi=[0, 100, 200, 300, 400, 500, 600],
             x=[-3, -2, -1, 0, 1, 2, 3])

# Part B & C - Using trainlm and trainbr algorithm

# Q2-2 === Trainlm Matlab

In [ ]:
# using trainlm

In [ ]:
Train_loss = [1.0149, 1.1406, 0.3574, 0.3475, 0.3841, 0.1060, 0.0155, 6.2299e-05, 0.0127, 1.1469e-05, 0.0155, 0.3382, 0.7501]
Test_loss = [0.9999, 1.1464, 0.3496, 0.3492, 0.3859, 0.1068, 0.0156, 3.1414e-05, 0.0129, 1.1649e-05, 0.0151, 0.3343, 0.6907]
hidden_net_sets = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 50, 100]

In [ ]:
plot_bars(Train_loss, 
          hidden_net_sets, 
          title='Train loss value of each neurons number (trainlm)',
          xlabel='Neurons',
          ylabel='Loss')

plot_bars(Test_loss, 
          hidden_net_sets, 
          title='Test loss value of each neurons number (trainlm)',
          xlabel='Neurons',
          ylabel='Loss')

# Q2-3 === Trainbr Matlab

In [ ]:
Train_loss = [1.0986, 1.1507, 1.2035, 1.0968, 0.1144, 1.1006, 4.0383e-04, 1.1905, 1.4337e-04, 5.1526e-07, 5.1346e-08, 7.2183e-11, 2.2767e-08]
Test_loss = [1.0995, 1.1518, 1.2091, 1.0977, 0.1154, 1.1010, 3.7432e-04, 1.1945, 1.2825e-04, 5.0316e-07, 1.5869e-07, 1.8761e-10, 1.3317e-08]

In [ ]:
plot_bars(Train_loss, 
          hidden_net_sets, 
          title='Train loss value of each neurons number (trainbr)',
          xlabel='Neurons',
          ylabel='Loss')

plot_bars(Test_loss, 
          hidden_net_sets, 
          title='Test loss value of each neurons number (trainbr)',
          xlabel='Neurons',
          ylabel='Loss')